# Destiny 2 ML practice project

Many thanks to Garrick Fernandez (https://github.com/garrickf) for guidance on code for Bungie API data collection!

This will be an exploration of various data science concepts, with no set plan as of now other than to explore the data available. A couple ideas could be: <br>
-Identifying hackers in trails  <br>
-Predicting which team will win in PvP  <br>
-Identify trends which increase player engagement eg. good trials weapon hypothesis testing??<br>

In [ ]:
import data_collection1.py

In [15]:
AllPvP = 5
Control = 10
Clash = 12
IronBanner=19
TrialsOfOsiris = 84

In [20]:
HEADERS = {"X-API-Key":os.environ["BUNGIE_API_KEY"]}
weapon = requests.get("https://www.bungie.net/platform/Destiny/Manifest/InventoryItem/1274330687/", headers=HEADERS)
inventoryItem = weapon.json()
print(inventoryItem['Response']['data']['inventoryItem']['itemName'])

Gjallarhorn


In [64]:
weapon = requests.get("https://www.bungie.net/platform/Destiny/Manifest/InventoryItem/1274330687/", headers=HEADERS)
inventoryItem = weapon.json()
print(inventoryItem['Response']['data']['inventoryItem']['itemName'])

Gjallarhorn


In [65]:
game = requests.get("https://www.bungie.net/platform/Destiny2/Stats/PostGameCarnageReport/84005478/", headers=HEADERS)
report = game.json()
report

{'Response': {'period': '2017-09-08T04:23:17Z',
  'startingPhaseIndex': 0,
  'activityWasStartedFromBeginning': False,
  'activityDetails': {'referenceId': 2266954523,
   'directorActivityHash': 2266954523,
   'instanceId': '84005478',
   'mode': 2,
   'modes': [7, 2],
   'isPrivate': False,
   'membershipType': 0},
  'entries': [{'standing': 0,
    'score': {'basic': {'value': 0.0, 'displayValue': '0'}},
    'player': {'destinyUserInfo': {'iconPath': '/common/destiny2_content/icons/ecd4608be3fb1c02755a96dc104dc003.jpg',
      'crossSaveOverride': 2,
      'applicableMembershipTypes': [3, 2],
      'isPublic': True,
      'membershipType': 2,
      'membershipId': '4611686018450981322',
      'displayName': 'rockerman91',
      'bungieGlobalDisplayName': 'rockerman91',
      'bungieGlobalDisplayNameCode': 4972},
     'characterClass': 'Titan',
     'classHash': 3655393761,
     'raceHash': 3887404748,
     'genderHash': 3111576190,
     'characterLevel': 20,
     'lightLevel': 283,
   

In [ ]:
try1 = requests.get("https://www.bungie.net/Platform/User/GetBungieNetUserById/1/",headers=HEADERS)
print(try1)
try1.json()

In [ ]:
try2 = requests.get("https://www.bungie.net/Platform/Trending/Categories/", headers=HEADERS)
try2.json()

In [ ]:
try3 = requests.get("https://www.bungie.net/Platform//HelloWorld/", headers=HEADERS)
try3.json()

In [ ]:
try4 = requests.get("https://www.bungie.net/Platform//GlobalAlerts/", headers=HEADERS)
try4.json()

In [ ]:
try5 = requests.get("https://bungie.net/Platform/Destiny2/Stats/PostGameCarnageReport/" + str(7234123546),headers=HEADERS)
try5.json()

In [ ]:
try6 = requests.get("https://bungie.net/Platform/Destiny2/SearchDestinyPlayer/All/" + 'newksie', headers=HEADERS)
try6.json()

In [ ]:
try7 = requests.get("https://www.bungie.net/Platform/User/Search/Prefix/newksie/0/", headers=HEADERS)
mymembershipId = try7.json()['Response']['searchResults'][0]['destinyMemberships'][0]['membershipId']
mymembershipId
#try8 = requests.get("https://www.bungie.net/Platform/Destiny2/3/Profile/"+str(mymembershipId)+'/',headers=HEADERS)
#try8.json()

In [105]:
import json
import logging
import os
import pickle
import time
from os.path import dirname, join, normpath, realpath
from urllib.parse import urljoin

import pandas as pd
import requests
from dotenv import load_dotenv

def get_director_activity_name(activity):
    # Uses the directorActivityHash of the
    # HistoricalStats.DestinyHistoricalStatsActivity (i.e., the entries of the
    # activity history)
    activity_hash = activity["activityDetails"]["directorActivityHash"]

    # Localize using manifest
    return INDEX["DestinyActivityDefinition"][activity_hash]["displayProperties"][
        "name"
    ]


def get_activity_period(activity):
    # TODO may want to parse datetime string or use for certain things
    return activity["period"]


def get_activity_reference_id(activity):
    # NOTE more properly thought of as the activityHash. Maps to
    # DestinyActivityDefinition in manifest.
    return activity["activityDetails"]["referenceId"]


def get_activity_instance_id(activity):
    # Used for looking up post-game carnage reports (PGCRs)
    return activity["activityDetails"]["instanceId"]


def get_activity_values(activity):
    return activity["values"]


def get_value_kills(values):
    if "kills" in values:
        return values["kills"]["basic"]["value"]
    else:
        return None


def get_value_deaths(values):
    if "deaths" in values:
        return values["deaths"]["basic"]["value"]
    else:
        return None


def get_value_assists(values):
    # NOTE: takes in values object from activity summary
    if "assists" in values:
        return values["assists"]["basic"]["value"]
    else:
        return None


def get_value_score(values):
    if "score" in values:
        return values["score"]["basic"]["value"]
    else:
        return None


def get_value_completed(values):
    # 1 if completed, NOTE haven't seen not completed but assuming it's 0
    if "completed" in values:
        return values["completed"]["basic"]["value"]
    else:
        return None


def get_value_opponents_defeated(values):
    # TODO what is the difference between this and kills?
    if "opponentsDefeated" in values:
        return values["opponentsDefeated"]["basic"]["value"]
    else:
        return None


def get_value_combat_efficiency(values):
    if "efficiency" in values:
        return values["efficiency"]["basic"]["value"]
    else:
        return None


def get_value_kdr(values):
    if "killsDeathsRatio" in values:
        return values["killsDeathsRatio"]["basic"]["value"]
    else:
        return None


def get_value_kda(values):
    if "killsDeathsAssists" in values:
        return values["killsDeathsAssists"]["basic"]["value"]
    else:
        return None


def get_value_activity_duration(values, in_units_of="seconds"):
    # Returns activity duration in seconds
    # TODO allow translation to other units if need be
    if "activityDurationSeconds" in values:
        return values["activityDurationSeconds"]["basic"]["value"]
    else:
        return None


def get_value_team(values):
    if "team" in values:
        return values["team"]["basic"]["value"]
    else:
        return None


def get_value_standing(values):
    # The API marks 1 as defeat, 0 as victory; we invert that
    if "standing" in values:
        return 1 - values["standing"]["basic"]["value"]
    else: 
        return None


def get_value_team_score(values):
    if "teamScore" in values:
        return values["teamScore"]["basic"]["value"]
    else:
        return None


def get_activity_history(character_data, params=None):
    path = urljoin(API_URL, GET_ACTIVITY_HISTORY.format(**character_data))
    r = requests.get(path, headers=HEADERS, params=params)

    activities = r.json()["Response"]["activities"]
    print(f"{len(activities)} activities found")

    history = []

    for activity in activities:
        period = get_activity_period(activity)
        details = activity["activityDetails"]
        activity_name = get_director_activity_name(activity)
        values = get_activity_values(activity)
        id = get_activity_instance_id(activity)

        # TODO: filter and bucket activities. get more. use recent activities as features per player
        # print(f"{period}, {activity_name}, victory?: {get_value_standing(values)}, kills: {get_value_kills(values)}, oppDef: {get_value_opponents_defeated(values)}")
        print(f"{period}, {activity_name}, {id}")
        history.append(
            {
                "period": period,
                "activity_name": activity_name,
                "details": details,
                "values": values,
            }
        )

    return history

In [107]:
STARTING_ACTIVITY_ID = 8400554258
ENDING_ACTIVITY_ID = STARTING_ACTIVITY_ID + int(10)  # 100K

API_URL = "https://www.bungie.net/Platform/"
GET_ACTIVITY_HISTORY = "Destiny2/{membershipType}/Account/{destinyMembershipId}/Character/{characterId}/Stats/Activities/"
GET_POST_GAME_CARNAGE_REPORT = "Destiny2/Stats/PostGameCarnageReport/{activityId}/"

HEADERS = {"X-API-Key":os.environ["BUNGIE_API_KEY"]}

In [108]:
instance_id=STARTING_ACTIVITY_ID+2000000
path = urljoin(API_URL, GET_POST_GAME_CARNAGE_REPORT.format(**{"activityId": instance_id}))
r = requests.get(path, headers=HEADERS, timeout=1)
pgcr = r.json()["Response"]

In [109]:
#pgcr["activityDetails"]["mode"]
#pgcr

In [110]:
def single_scrape(activity_id):
    path = urljoin(API_URL, GET_POST_GAME_CARNAGE_REPORT.format(**{"activityId": activity_id}))
    r = requests.get(path, headers=HEADERS)
    #r = requests.get(path, headers=HEADERS, timeout=1)
    pgcr = r.json()["Response"]
    
    period = get_activity_period(pgcr)
    director_activity_mode = pgcr["activityDetails"]["mode"]
    
    entries = pgcr["entries"]
    # print(f"Activity {instance_id} has {len(entries)} entries...")

    player_entries = {}
    for i, entry in enumerate(entries):
        # TODO handle extended and other props
        characterId = entry["characterId"]
        player_entries[f"player{i + 1}_char_id"] = characterId

        # Values (kills, assists, etc.)
        if "values" not in entry: 
            continue
        
        values = entry["values"]
        player_entries[f"player{i + 1}_kills"] = get_value_kills(values)
        player_entries[f"player{i + 1}_deaths"] = get_value_deaths(values)
        player_entries[f"player{i + 1}_assists"] = get_value_assists(values)
        player_entries[f"player{i + 1}_kdr"] = get_value_kdr(values)
        player_entries[f"player{i + 1}_kda"] = get_value_kda(values)
        player_entries[f"player{i + 1}_efficiency"] = get_value_combat_efficiency(
            values
        )
        player_entries[f"player{i + 1}_score"] = get_value_score(values)

        player_entries[f"player{i + 1}_standing"] = get_value_standing(values)
        player_entries[f"player{i + 1}_team_score"] = get_value_team_score(values)
        player_entries[
            f"player{i + 1}_activity_duration"
        ] = get_value_activity_duration(values)

    # Return dict
    return {
        "instance_id": instance_id,
        "period": period,
        "director_activity_mode": director_activity_mode,
        **player_entries,
    }

In [111]:
def multi_scrape(start, end, mode):
    dataset=[]
    for activity_id in range(start,end):
        data_point = single_scrape(activity_id)
        if data_point["director_activity_mode"] == mode:
            dataset.append(data_point)
    df = pd.DataFrame(dataset)
    print(df)
    #df.to_csv("test.csv")

In [112]:
multi_scrape(STARTING_ACTIVITY_ID+1000,STARTING_ACTIVITY_ID+1005, 5)

Empty DataFrame
Columns: []
Index: []
